In [ ]:
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn import metrics


letters = pd.read_csv('./letter_data/letter-recognition.txt')

training_points = np.array(letters[:15000].drop(['letter'], 1))
training_labels = np.array(letters[:15000]['letter'])

clf = tree.DecisionTreeClassifier()
clf = clf.fit(training_points,training_labels)


test_points = np.array(letters[1500:].drop(['letter'], 1))
test_labels = np.array(letters[1500:]['letter'])

accuracy = clf.score(test_points, test_labels)

print(float(accuracy))

expected = test_labels
predicted = clf.predict(test_points)

# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

<ipython-input-2-3e92aaa639fe>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  training_points = np.array(letters[:15000].drop(['letter'], 1))
<ipython-input-2-3e92aaa639fe>:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_points = np.array(letters[1500:].drop(['letter'], 1))


0.9648648648648649
              precision    recall  f1-score   support

           A       0.99      0.98      0.98       729
           B       0.93      0.97      0.95       702
           C       0.97      0.96      0.97       685
           D       0.95      0.97      0.96       736
           E       0.95      0.95      0.95       714
           F       0.96      0.96      0.96       728
           G       0.97      0.95      0.96       716
           H       0.93      0.93      0.93       667
           I       0.98      0.97      0.97       700
           J       0.97      0.97      0.97       672
           K       0.96      0.96      0.96       680
           L       0.98      0.98      0.98       709
           M       0.98      0.98      0.98       718
           N       0.97      0.96      0.97       726
           O       0.96      0.97      0.96       702
           P       0.95      0.96      0.96       736
           Q       0.96      0.96      0.96       734
        

In [ ]:
import numpy as np
import pandas as pd
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split

# Read and preprocess data
letters = pd.read_csv('./letter_data/letter-recognition.txt')
X = letters.drop(['letter'], axis=1)
y = letters['letter']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define fitness function
def evaluate(max_depth, min_samples_split, X_train, y_train, X_test, y_test):
    clf = tree.DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split)
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    return accuracy

# Evolutionary algorithm parameters
POP_SIZE = 10
NUM_GENERATIONS = 100
MUTATION_RATE = 0.2

# Initialize population
population = np.random.randint(2, 20, size=(POP_SIZE, 2))

# Main loop
for gen in range(NUM_GENERATIONS):
    # Evaluate fitness
    fitnesses = [evaluate(max_depth, min_samples_split, X_train, y_train, X_test, y_test) for max_depth, min_samples_split in population]

    # Select top individuals
    sorted_indices = np.argsort(fitnesses)[::-1]
    top_individuals = population[sorted_indices[:2]]

    # Generate new population
    new_population = np.zeros_like(population)
    for i in range(POP_SIZE):
        parent1, parent2 = np.random.choice(2, 2, replace=False)
        offspring = np.array([top_individuals[parent1, 0], top_individuals[parent2, 1]])
        
        # Mutation
        if np.random.rand() < MUTATION_RATE:
            mutation_idx = np.random.randint(2)
            mutation_value = np.random.randint(-2, 3)
            offspring[mutation_idx] += mutation_value
            offspring[mutation_idx] = max(2, offspring[mutation_idx])

        new_population[i] = offspring

    population = new_population
    best_individual = top_individuals[0]
    print(f"Generation {gen + 1}: Best fitness = {max(fitnesses)}, max_depth = {best_individual[0]}, min_samples_split = {best_individual[1]}")

# Train and evaluate the best decision tree
best_max_depth, best_min_samples_split = best_individual
best_clf = tree.DecisionTreeClassifier(max_depth=best_max_depth, min_samples_split=best_min_samples_split)
best_clf.fit(X_train, y_train)

accuracy = best_clf.score(X_test, y_test)
print(f"Final accuracy: {accuracy}")

expected = y_test
predicted = best_clf.predict(X_test)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


Generation 1: Best fitness = 0.8616666666666667, max_depth = 19, min_samples_split = 6
Generation 2: Best fitness = 0.8616666666666667, max_depth = 19, min_samples_split = 6
Generation 3: Best fitness = 0.8631666666666666, max_depth = 19, min_samples_split = 6
Generation 4: Best fitness = 0.8628333333333333, max_depth = 19, min_samples_split = 6
Generation 5: Best fitness = 0.8636666666666667, max_depth = 19, min_samples_split = 6
Generation 6: Best fitness = 0.8623333333333333, max_depth = 19, min_samples_split = 6
Generation 7: Best fitness = 0.8631666666666666, max_depth = 19, min_samples_split = 6
Generation 8: Best fitness = 0.8623333333333333, max_depth = 19, min_samples_split = 6
Generation 9: Best fitness = 0.862, max_depth = 19, min_samples_split = 6
Generation 10: Best fitness = 0.8635, max_depth = 19, min_samples_split = 6
Generation 11: Best fitness = 0.8663333333333333, max_depth = 19, min_samples_split = 5
Generation 12: Best fitness = 0.8665, max_depth = 19, min_samples_

In [ ]:
import numpy as np
import pandas as pd
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split

# Read and preprocess data
letters = pd.read_csv('./letter_data/letter-recognition.txt')
X = letters.drop(['letter'], axis=1)
y = letters['letter']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define fitness function
def evaluate(clf, X_test, y_test):
    accuracy = clf.score(X_test, y_test)
    return accuracy

# Create decision tree classifier
def create_classifier(max_depth, min_samples_split):
    clf = tree.DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split)
    clf.fit(X_train, y_train)
    return clf

# Evolutionary algorithm parameters
POP_SIZE = 10
NUM_GENERATIONS = 50
MUTATION_RATE = 0.2

# Initialize population
population = [create_classifier(np.random.randint(2, 20), np.random.randint(2, 20)) for _ in range(POP_SIZE)]

# Main loop
for gen in range(NUM_GENERATIONS):
    # Evaluate fitness
    fitnesses = [evaluate(clf, X_test, y_test) for clf in population]

    # Select top individuals
    sorted_indices = np.argsort(fitnesses)[::-1]
    top_individuals = [population[i] for i in sorted_indices[:2]]

    # Generate new population
    new_population = []
    for i in range(POP_SIZE):
        parent1, parent2 = np.random.choice(2, 2, replace=False)
        offspring_max_depth = top_individuals[parent1].get_params()['max_depth']
        offspring_min_samples_split = top_individuals[parent2].get_params()['min_samples_split']

        # Mutation
        if np.random.rand() < MUTATION_RATE:
            mutation_idx = np.random.randint(2)
            mutation_value = np.random.randint(-2, 3)
            if mutation_idx == 0:
                offspring_max_depth += mutation_value
                offspring_max_depth = max(2, offspring_max_depth)
            else:
                offspring_min_samples_split += mutation_value
                offspring_min_samples_split = max(2, offspring_min_samples_split)

        offspring = create_classifier(offspring_max_depth, offspring_min_samples_split)
        new_population.append(offspring)

    population = new_population
    best_individual = top_individuals[0]
    best_hyperparams = best_individual.get_params()
    print(f"Generation {gen + 1}: Best fitness = {max(fitnesses)}, max_depth = {best_hyperparams['max_depth']}, min_samples_split = {best_hyperparams['min_samples_split']}")

# Evaluate the best decision tree
accuracy = evaluate(best_individual, X_test, y_test)
print(f"Final accuracy: {accuracy}")

expected = y_test
predicted = best_individual.predict(X_test)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


Generation 1: Best fitness = 0.801, max_depth = 14, min_samples_split = 14
Generation 2: Best fitness = 0.8045, max_depth = 14, min_samples_split = 12
Generation 3: Best fitness = 0.824, max_depth = 16, min_samples_split = 12
Generation 4: Best fitness = 0.8306666666666667, max_depth = 16, min_samples_split = 10
Generation 5: Best fitness = 0.8391666666666666, max_depth = 17, min_samples_split = 10
Generation 6: Best fitness = 0.8405, max_depth = 17, min_samples_split = 10
Generation 7: Best fitness = 0.8456666666666667, max_depth = 17, min_samples_split = 8
Generation 8: Best fitness = 0.8535, max_depth = 19, min_samples_split = 8
Generation 9: Best fitness = 0.8581666666666666, max_depth = 21, min_samples_split = 8
Generation 10: Best fitness = 0.8595, max_depth = 21, min_samples_split = 8
Generation 11: Best fitness = 0.8595, max_depth = 21, min_samples_split = 8
Generation 12: Best fitness = 0.8595, max_depth = 22, min_samples_split = 8
Generation 13: Best fitness = 0.8595, max_dep

In [2]:
# SVM
import numpy as np
from sklearn.svm import SVC
import pandas as pd
from sklearn import metrics

letters = pd.read_csv('letter-recognition.txt')

training_points = np.array(letters[:15000].drop(['letter'], 1))
training_labels = np.array(letters[:15000]['letter'])

clf = SVC()
clf.fit(training_points, training_labels) 


test_points = np.array(letters[15000:].drop(['letter'], 1))
test_labels = np.array(letters[15000:]['letter'])

expected = test_labels
predicted = clf.predict(test_points)

accuracy = clf.score(test_points, test_labels)

print(float(accuracy))

# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

<ipython-input-2-391b521225eb>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  training_points = np.array(letters[:15000].drop(['letter'], 1))
<ipython-input-2-391b521225eb>:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_points = np.array(letters[15000:].drop(['letter'], 1))


0.916
              precision    recall  f1-score   support

           A       0.97      0.96      0.97       206
           B       0.81      0.94      0.87       173
           C       0.94      0.89      0.92       171
           D       0.83      0.94      0.88       216
           E       0.85      0.90      0.87       191
           F       0.88      0.94      0.91       194
           G       0.86      0.90      0.88       208
           H       0.86      0.81      0.83       178
           I       0.96      0.91      0.94       205
           J       0.95      0.91      0.93       183
           K       0.89      0.90      0.90       177
           L       0.99      0.88      0.93       205
           M       0.93      0.96      0.95       187
           N       0.96      0.90      0.93       198
           O       0.87      0.90      0.88       181
           P       0.98      0.84      0.90       207
           Q       0.94      0.94      0.94       217
           R       0.

In [3]:
# naive bayes
import numpy as np
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn import metrics


letters = pd.read_csv('letter-recognition.txt')

training_points = np.array(letters[:15000].drop(['letter'], 1))
training_labels = np.array(letters[:15000]['letter'])


clf = GaussianNB()
clf.fit(training_points, training_labels)


test_points = np.array(letters[15000:].drop(['letter'], 1))
test_labels = np.array(letters[15000:]['letter'])
# predicts = clf.predict(test_points)

accuracy = clf.score(test_points, test_labels)

print(float(accuracy))

expected = test_labels
predicted = clf.predict(test_points)

# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))



<ipython-input-3-ce5acfcd2279>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  training_points = np.array(letters[:15000].drop(['letter'], 1))
<ipython-input-3-ce5acfcd2279>:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_points = np.array(letters[15000:].drop(['letter'], 1))


0.633
              precision    recall  f1-score   support

           A       0.85      0.85      0.85       206
           B       0.45      0.70      0.55       173
           C       0.74      0.74      0.74       171
           D       0.57      0.69      0.62       216
           E       0.67      0.39      0.49       191
           F       0.69      0.73      0.71       194
           G       0.57      0.54      0.55       208
           H       0.50      0.30      0.37       178
           I       0.54      0.75      0.63       205
           J       0.76      0.75      0.75       183
           K       0.43      0.45      0.44       177
           L       0.95      0.76      0.84       205
           M       0.67      0.91      0.78       187
           N       0.86      0.60      0.71       198
           O       0.46      0.68      0.55       181
           P       0.86      0.72      0.79       207
           Q       0.59      0.53      0.56       217
           R       0.